In [165]:
# inputs
presetName = 'Liquid_Light-Multi'
maxThrust = 67
pitch = 1.0
# for Multi
secondaryMaxThrust = 160

isMulti = presetName.__contains__("Multi")

# x1 = key min, x2 = key max, y1 = value min, y2 = value max, input
def lerp(x1, x2, y1, y2, input):
    output = y1 + (((input - x1) * (y2 - y1)) / (x2 - x1))
    return round(output,2) 

# Engine Samples and their loudness in LUFS with pitch change accounted for
# Sample Name : 0.5 pitch , 1.0 pitch, 2.0 pitch
sampleLoudnessMap = {
    'Liquid_Heavy-Atm':     [-7.20,	-5.40, -4.50],
    'Liquid_Heavy-Vac':     [-7.10,	-5.30, -4.40],
    'Liquid_Medium-Atm':    [-9.50,	-7.70, -6.80],
    'Liquid_Medium-Vac':    [-9.70,	-7.90, -7.00],
    'Liquid_Light-Atm':     [-11.40,-9.90, -9.00],
    'Liquid_Light-Vac':     [-11.50, -10.00, -9.10],
    'SRB_Heavy':            [-6.00, -4.60, -3.80],
    'SRB_Medium':           [-6.50, -5.40, -4.50],
    'SRB_Light':            [-8.10, -7.10, -6.00],
}

# Engine Presets
presets = {
    'Liquid_Heavy-Lifter'       : sampleLoudnessMap['Liquid_Heavy-Atm'],
    'Liquid_Heavy-Sustainer'    : sampleLoudnessMap['Liquid_Heavy-Vac'],
    'Liquid_Heavy-Vacuum'       : sampleLoudnessMap['Liquid_Heavy-Vac'],
    'Liquid_Medium-Lifter'      : sampleLoudnessMap['Liquid_Medium-Atm'],
    'Liquid_Medium-Sustainer'   : sampleLoudnessMap['Liquid_Medium-Vac'],
    'Liquid_Medium-Vacuum'      : sampleLoudnessMap['Liquid_Medium-Vac'],
    'Liquid_Light-Atmosphere'   : sampleLoudnessMap['Liquid_Light-Atm'],
    'Liquid_Light-Atmosphere_1' : sampleLoudnessMap['Liquid_Light-Atm'],
    'Liquid_Light-Vacuum'       : sampleLoudnessMap['Liquid_Light-Vac'],
    'Liquid_Light-Vacuum_1'     : sampleLoudnessMap['Liquid_Light-Vac'],
    'SRB_Heavy'                 : sampleLoudnessMap['SRB_Heavy'],
    'SRB_Medium'                : sampleLoudnessMap['SRB_Medium'],
    'SRB_Light'                 : sampleLoudnessMap['SRB_Light'],
    'SRB_Light_1'               : sampleLoudnessMap['SRB_Light'],
}

# Engine Multi Presets. Engine 1: Vac, Engine 2: Atm
presetsMulti = {
    'Liquid_Heavy-Multi'    : [sampleLoudnessMap['Liquid_Heavy-Vac'], sampleLoudnessMap['Liquid_Heavy-Atm']],
    'Liquid_Medium-Multi'   : [sampleLoudnessMap['Liquid_Medium-Vac'], sampleLoudnessMap['Liquid_Medium-Atm']],
    'Liquid_Light-Multi'    : [sampleLoudnessMap['Liquid_Light-Vac'], sampleLoudnessMap['Liquid_Light-Atm']],
    'Liquid_Light-Multi_1'  : [sampleLoudnessMap['Liquid_Light-Vac'], sampleLoudnessMap['Liquid_Light-Atm']],
}

volume = 1
rawLoudness = -10
rawLoudnessAdjusted = -10

secondaryVolumeScale = 1.0
secondaryRawLoudness = -10
secondaryRawLoudnessAdjusted = -10

if maxThrust < 1000:
    rawLoudness = lerp(100, 1000, -10, -5, maxThrust)
else:
    rawLoudness = lerp(1000, 5000, -5, 1, maxThrust)

if isMulti:
    if secondaryMaxThrust < 1000:
        secondaryRawLoudness = lerp(100, 1000, -10, -5, secondaryMaxThrust)
    else:
        secondaryRawLoudness = lerp(1000, 5000, -5, 1, secondaryMaxThrust)
        
    if pitch < 1:
        rawLoudnessAdjusted = lerp(0.5, 1.0, presetsMulti[presetName][0][0], presetsMulti[presetName][0][1], pitch)
        secondaryRawLoudnessAdjusted = lerp(0.5, 1.0, presetsMulti[presetName][1][0], presetsMulti[presetName][1][1], pitch)
    else:
        rawLoudnessAdjusted = lerp(1.0, 2.0, presetsMulti[presetName][0][1], presetsMulti[presetName][0][2], pitch)
        secondaryRawLoudnessAdjusted = lerp(1.0, 2.0, presetsMulti[presetName][1][1], presetsMulti[presetName][1][2], pitch)
    
    volume = round(pow(10, (rawLoudness/20)) / pow(10, (rawLoudnessAdjusted/20)), 2)
    volume2 = round(pow(10, (secondaryRawLoudness/20)) / pow(10, (secondaryRawLoudnessAdjusted/20)), 2)
    secondaryVolumeScale = round(volume2 / volume, 2)

else:
    if pitch < 1:
        rawLoudnessAdjusted = lerp(0.5, 1.0, presets[presetName][0], presets[presetName][1], pitch)
    else:
        rawLoudnessAdjusted = lerp(1.0, 2.0, presets[presetName][1], presets[presetName][2], pitch)
    volume = round(pow(10, (rawLoudness/20)) / pow(10, (rawLoudnessAdjusted/20)), 2)


print("//   maxThrust:", maxThrust, "kN", "| Loudness:", rawLoudness, "LUFS")
if isMulti:
    print("//   secondaryMaxThrust:", secondaryMaxThrust, "kN", "| Loudness:", secondaryRawLoudness, "LUFS")
print("RSE_PRESET\n{")
print("     name =", presetName)
print("     volume =", volume)
print("     pitch =", pitch)
if presetName.__contains__("Multi"):
    print("     secondaryEngine_VolumeScale =", secondaryVolumeScale)
print("}")


//   maxThrust: 67 kN | Loudness: -10.18 LUFS
//   secondaryMaxThrust: 160 kN | Loudness: -9.67 LUFS
RSE_PRESET
{
     name = Liquid_Light-Multi
     volume = 0.98
     pitch = 1.0
     secondaryEngine_VolumeScale = 1.05
}
